In [4]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
is_out_exist = os.path.exists('out')
if not is_out_exist:
    os.makedirs('out')
    print("The new directory is created!")
reference_image_name = '.\\img\\40.jpg'
target_image_name    = '.\\img\\42.jpg'

In [7]:
def generate_search_window(i,j,p, shape):
    lu = (max(0, i-p-1), max(0, j-p-1))               #i-p, j-p
    ld = (min(shape[0], i+p), max(0, j-p-1))        #i+p, j-p
    ru = (max(0, i-p-1), min(shape[1], j+p))        #i-p, j+p
    rd = (min(shape[0], i+p), min(shape[1], j+p)) #i+p, j+p
    return lu, ld, ru, rd
def SAD(window, blk):
    return np.sum(np.abs(window-blk))
def full_search(window_img, target_img, N):
    #print(window_img.shape, target_img.shape)
    min_i = 0
    min_j = 0
    min_value = 1000000000000
    for i in range(window_img.shape[0]-N):
        for j in range(window_img.shape[1]-N):
            tmp = SAD(window_img[i:i+N,j:j+N,:], target_img)
            if tmp < min_value:
                min_value = tmp
                min_i = i
                min_j = j
    return min_i, min_j
def motion_estimation(reference_image_name, target_image_name, N, p):
    reference_image = cv2.imread(reference_image_name, cv2.IMREAD_COLOR)
    target_image = cv2.imread(target_image_name, cv2.IMREAD_COLOR)
    motion_vectors = np.zeros((int(reference_image.shape[0]/N), int(reference_image.shape[1]/N), 2))
    residual_image = np.zeros((reference_image.shape[0], reference_image.shape[1], 3))
    predicted_image = np.zeros((reference_image.shape[0], reference_image.shape[1], 3))
    for i in range(0, target_image.shape[0], N):
        for j in range(0, target_image.shape[1], N):
            blk = target_image[i:i+N, j:j+N]
            lu, ld, ru, rd = generate_search_window(int((i+i+N)/2),int((j+j+N)/2), p, reference_image.shape)
            window = reference_image[lu[0]: ld[0], lu[1]: rd[1]]
            min_i, min_j = full_search(window, blk, N)
            min_i = min_i + lu[0]
            min_j = min_j + lu[1]
            motion_vectors[int(i/N), int(j/N)] = (int(min_i-i), int(min_j-j))        
            residual_image[i:i+N, j:j+N] = blk - reference_image[min_i:min_i+N, min_j:min_j+N, : ]
            predicted_image[i:i+N, j:j+N] = reference_image[i+int(min_i-i):i+int(min_i-i)+N, j+int(min_j-j):j+int(min_j-j)+N, : ]
    return motion_vectors, residual_image, predicted_image
def draw_arrow_line(estimated_motion, reference_image_name, N, vector_image_name):
    img = cv2.imread(reference_image_name, cv2.IMREAD_COLOR)
    for i in range(0,img.shape[0], N):
        for j in range(0,img.shape[1], N):
            start = (i,j)
            end = (int(i+estimated_motion[int(i/N), int(j/N)][0]),int(j + estimated_motion[int(i/N), int(j/N)][1]))
            
            start = (start[1], start[0])
            end = (end[1], end[0])
            cv2.arrowedLine(img, start, end, (0,0,225))
    cv2.imwrite(vector_image_name, img)
    cv2.imshow('arrow line image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def draw_residual(residual_image, residul_image_name):
    cv2.imwrite(residul_image_name, residual_image)
    cv2.imshow('residual image', residual_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def draw_predicted_image(predicted_image, predicted_image_name):
    cv2.imwrite(predicted_image_name, predicted_image)
    cv2.imshow('predicted image', predicted_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
for approach in ['full']:
    for N in [8]:
        for p in [8]:
            estimated_motion, residual_image, predicted_image = motion_estimation(reference_image_name=reference_image_name, target_image_name=target_image_name, N=N, p=p)
            #print(estimated_motion.shape)
            #print(np.std(estimated_motion), np.mean(estimated_motion))
            vector_image_name = 'out\\'+approach+'_motion_vection_r'+str(N)+'_p'+str(p)+'.png'
            residual_image_name = 'out\\'+approach+'_residual_r'+str(N)+'_p'+str(p)+'_residual.png'
            predicted_image_name = 'out\\'+approach+'_predicted_r'+str(N)+'_p'+str(p)+'_predicted.png'
            draw_predicted_image(predicted_image, predicted_image_name=predicted_image_name)
            draw_residual(residual_image, residul_image_name=residual_image_name)
            draw_arrow_line(estimated_motion, target_image_name, N=N, vector_image_name=vector_image_name)

(90, 160, 2)
2.8680949517656753 -1.1903125
